## notebook setup and configuration

In [31]:
import numpy as np
import pandas as pd
import requests
import json

In [32]:
# load api keys from api_keys.txt file
mbta_api = 'https://api-v3.mbta.com/'

api_keys = dict()
with open('api_keys.txt', 'r') as f:
    for line in f.readlines():
        name, key = line.split('=')
        api_keys[name] = key.rstrip()

In [33]:
# setup google maps
import gmaps
import gmaps.datasets
gmaps.configure(api_key=api_keys['gmaps_api_key'])

In [4]:
r = requests.get(mbta_api+'routes/')
r.status_code

200

In [5]:
r.headers['content-type']
route_data = r.json()['data']

In [6]:
route_1 = [d for d in route_data if d['id'] == '1']
route_1_link = route_1[0]['links']['self']
route_1_data = requests.get('https://api-v3.mbta.com/routes/1')
route_1_data.json()

{'data': {'attributes': {'color': 'FFC72C',
   'description': 'Key Bus Route (Frequent Service)',
   'direction_names': ['Outbound', 'Inbound'],
   'long_name': '',
   'short_name': '1',
   'sort_order': 100,
   'text_color': '000000',
   'type': 3},
  'id': '1',
  'links': {'self': '/routes/1'},
  'type': 'route'},
 'jsonapi': {'version': '1.0'}}

# current train map

In [47]:
# get line names and colors
train_line_ids = ['Red', 'Orange', 'Blue', 'Green-B', 'Green-C', 'Green-D', 'Green-E']
train_route_data = [r['attributes'] for r in route_data if r['id'] in train_line_ids]
trd = {key:value for key, value in zip(train_line_ids, train_route_data)}

{'Red': {'color': 'DA291C',
  'description': 'Rapid Transit',
  'direction_names': ['Southbound', 'Northbound'],
  'long_name': 'Red Line',
  'short_name': '',
  'sort_order': 1,
  'text_color': 'FFFFFF',
  'type': 1},
 'Orange': {'color': 'ED8B00',
  'description': 'Rapid Transit',
  'direction_names': ['Southbound', 'Northbound'],
  'long_name': 'Orange Line',
  'short_name': '',
  'sort_order': 2,
  'text_color': 'FFFFFF',
  'type': 1},
 'Blue': {'color': '00843D',
  'description': 'Rapid Transit',
  'direction_names': ['Westbound', 'Eastbound'],
  'long_name': 'Green Line B',
  'short_name': 'B',
  'sort_order': 3,
  'text_color': 'FFFFFF',
  'type': 0},
 'Green-B': {'color': '00843D',
  'description': 'Rapid Transit',
  'direction_names': ['Westbound', 'Eastbound'],
  'long_name': 'Green Line C',
  'short_name': 'C',
  'sort_order': 4,
  'text_color': 'FFFFFF',
  'type': 0},
 'Green-C': {'color': '00843D',
  'description': 'Rapid Transit',
  'direction_names': ['Westbound', 'Eastb

In [34]:
def query_vehicles():
    vehicle_request = requests.get(mbta_api+'vehicles/').json()['data']
    return vehicle_request

def line_vehicles(vehicle_data, line_id):
    return [v for v in vehicle_data if v['relationships']['route']['data']['id'] == line_id]

def vehicle_lat_lon(vehicles):
    return [(v['attributes']['latitude'], v['attributes']['longitude']) for v in vehicles ]

vehicle_data = query_vehicles()
red_line_trains = line_vehicles(vehicle_data, line_id='Red')
red_line_trains[0]

{'attributes': {'bearing': 200.0,
  'current_status': 'IN_TRANSIT_TO',
  'current_stop_sequence': 130,
  'direction_id': 0,
  'label': '1502',
  'last_updated': '2018-04-29T18:26:29-04:00',
  'latitude': 42.32915115356445,
  'longitude': -71.05667877197266,
  'speed': None},
 'id': 'R-54554657',
 'links': {'self': '/vehicles/R-54554657'},
 'relationships': {'route': {'data': {'id': 'Red', 'type': 'route'}},
  'stop': {'data': {'id': '70085', 'type': 'stop'}},
  'trip': {'data': {'id': 'ADDED-1524988862', 'type': 'trip'}}},
 'type': 'vehicle'}

In [35]:
red_line_lat_lon = np.asarray(vehicle_lat_lon(red_line_trains))
rltc_array = red_line_lat_lon
rltc_array

array([[ 42.32915115, -71.05667877],
       [ 42.38388062, -71.11978912],
       [ 42.35649872, -71.0626297 ],
       [ 42.3162384 , -71.0521698 ],
       [ 42.29320145, -71.06585693],
       [ 42.25830078, -71.0116806 ],
       [ 42.29668045, -71.0512619 ],
       [ 42.24995041, -71.00434113],
       [ 42.32051849, -71.05223083],
       [ 42.3670311 , -71.10626984],
       [ 42.36243057, -71.0858078 ],
       [ 42.35145187, -71.05552673],
       [ 42.3961792 , -71.14054108]])

plot the train locations on a map

In [36]:
# create a gmaps symbol layer of red dots
trans_red = "rgba(200, 0, 0, 0.4)"
red_line_dots = gmaps.symbol_layer(
    rltc_array, fill_color=trans_red,
    stroke_color=trans_red, scale=3
)

In [37]:
figure_layout = {
    'width':'500px',
    'height':'500px'
}

In [38]:
fig=gmaps.figure(layout=figure_layout)
fig.add_layer(red_line_dots)
fig.add_layer(gmaps.transit_layer())
fig

Figure(layout=FigureLayout(height='500px', width='500px'))